In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [2]:
prompts_path = "Prompts/fr/"

In [5]:
with open(prompts_path+'prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Description du contexte de l'entreprise.")

    def __str__(self):
        return self.context

res = get_analysis(prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Darty est une chaîne de magasins française spécialisée dans la distribution de matériel électroménager, de produits informatiques et audiovisuels (TV, audio, photo). Fondée en 1957, l'entreprise a évolué au fil des années, devenant un acteur majeur dans son secteur en France. Récemment, Darty a continué à se développer en renforçant sa présence en ligne et en élargissant son offre de services, notamment avec des options de livraison améliorées et des services après-vente étendus. Elle fait face à une concurrence accrue de la part de grandes enseignes telles que Boulanger, Fnac, et des géants du e-commerce comme Amazon. Darty s'efforce de maintenir sa position sur le marché en mettant l'accent sur la qualité de service et l'expérience client.


In [6]:
async def get_async_analysis(prompt, response_model):
    response: response_model = await client.chat.completions.create(
        messages=[
            {"role": "system", "content": "Tu est un assistant spélialisé dans l'analyse de commentaires, et qui ne renvoit que des fichiers JSON."},
            {"role": "user", "content": str(prompt)},
        ],
        response_format={ "type": "json_object" },
        model=GENERATION_ENGINE,
        temperature=TEMPERATURE,
        #max_retries=MAX_RETRIES,
        response_model=response_model,
        )
    return response #.choices[0].message.content

def apply_async_analysis(prompts, response_models):
    if type(response_models) is not list:
        response_models = [response_models for _ in prompts]
    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(get_async_analysis(prompt, response_model)) for (prompt, response_model) in zip(prompts, response_models)]
    res =  loop.run_until_complete(asyncio.gather(*tasks))
    return res

def get_analysis(prompt, response_model):
    return apply_async_analysis([prompt], response_model)[0]


In [9]:
with open(prompts_path+'prompt_categories_creation.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)

class InsightSubCategory(BaseModel):
    name: str = Field(description="Nom de la sous-catégorie.")
    #description: str = Field(description="Phrase de description de la sous-catégorie.")
    #id: str
    def __str__(self):
        return '\n- ' + self.name #+ ": " + self.description

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    #description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[InsightSubCategory] = Field(description="Sous-catégories.")
    #id: str

    def __str__(self):
        return self.name + " : " + ''.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Liste des catégories parmi lesquels les insights pourront être classifiées.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(prompt_categories_creation, InsightCategories)
print(res)

Retrying, exception: 1 validation error for InsightCategories
  Invalid JSON: control character (\u0000-\u001F) found while parsing a string at line 114 column 0 [type=json_invalid, input_value='{\n  "categories": [\n  ...n\n\n\n\n\n\n\n\n\n\n\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/json_invalid
Traceback (most recent call last):
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/patch.py", line 246, in retry_async
    return await process_response_async(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/patch.py", line 207, in process_response_async
    model = await response_model.from_response_async(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/function_calls.py", line 221, in from_response_async
    re

AttributeError: 'InsightSubCategory' object has no attribute 'description'

Retrying, exception: 1 validation error for InsightCategories
  Invalid JSON: control character (\u0000-\u001F) found while parsing a string at line 114 column 0 [type=json_invalid, input_value='{\n  "categories": [\n  ...      }\n      \n      ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/json_invalid
Traceback (most recent call last):
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/patch.py", line 246, in retry_async
    return await process_response_async(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/patch.py", line 207, in process_response_async
    model = await response_model.from_response_async(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/function_calls.py", line 221, in from_response_async
    re

In [11]:
res.categories

[InsightCategory(name="Expérience d'achat", sub_categories=[InsightSubCategory(name='Site Web', id='site_web'), InsightSubCategory(name='Magasin physique', id='magasin_physique'), InsightSubCategory(name='Processus de commande', id='processus_commande'), InsightSubCategory(name='Disponibilité des produits', id='disponibilite_produits')], id='experience_achat'),
 InsightCategory(name='Livraison', sub_categories=[InsightSubCategory(name='Délais de livraison', id='delais_livraison'), InsightSubCategory(name='Condition du colis', id='condition_colis'), InsightSubCategory(name='Suivi de livraison', id='suivi_livraison')], id='livraison'),
 InsightCategory(name='Produit', sub_categories=[InsightSubCategory(name='Conformité', id='conformite'), InsightSubCategory(name='Qualité', id='qualite'), InsightSubCategory(name='Installation et mise en service', id='installation_service')], id='produit'),
 InsightCategory(name='Service après-vente', sub_categories=[InsightSubCategory(name='Support techni

In [ ]:
for categorie in tqdm(res.categories):
    categorie.id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
        })
    
    for sub_categorie in categorie.sub_categories:
        sub_categorie.id = bubble_client.create(
            "SubCategory",
            {
                "Company": COMPANY_ID,
                "Category": categorie.id,
                "Name": sub_categorie.name,
                "Project": PROJECT_ID,
                "Description": sub_categorie.description,
            })

  0%|          | 0/5 [00:00<?, ?it/s]